In [4]:
1e-6*100

9.999999999999999e-05

In [ ]:
0.0001

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
# from tensorflow.keras import layers
from tensorflow import keras 
from scipy.stats import norm, uniform
import tensorflow as tf
import numpy as np
%config Completer.use_jedi = False
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, GRU, TimeDistributed, Input, Masking


In [48]:
train_data = np.load("data/test_data.npy")
train_data = train_data/train_data[:,:,:1]

In [42]:
def trade_model():
    
    ### timeseries sub_network
    input_layer = keras.layers.Input(shape=(None,5,))
    hidden_layer = keras.layers.GRU(10, activation="tanh",return_sequences = False, name="gru1")(input_layer)
    output_layer = keras.layers.Dense(2, activation="relu", name ="dense2")(hidden_layer)
    
    
    ### higher level vinalla neural network
    input2 = keras.layers.Input(shape = (3))
    concat = keras.layers.Concatenate()([output_layer, input2])
    h2 = keras.layers.Dense(20,activation = "relu")(concat)
    h2 = keras.layers.Dense(20,activation = "relu")(h2)
    h2 = keras.layers.Dense(20,activation = "relu")(h2)
    h2 = keras.layers.Dense(20,activation = "relu")(h2)
    h2 = keras.layers.Dense(20,activation = "relu")(h2)
    out2 = keras.layers.Dense(11, activation = "linear")(h2)
     
    model = keras.Model(inputs = [input_layer, input2], outputs = [out2])
    return model


# def trade_model():
    
#     ### timeseries sub_network
#     input_layer = keras.layers.Input(shape=(None,5,))
#     hidden_layer = keras.layers.GRU(20, activation="tanh",return_sequences = True, name="gru1")(input_layer)
#     hidden_layer = keras.layers.GRU(20, activation = "tanh", return_sequences =False, name = "gru2")(hidden_layer)
#     hidden_layer = keras.layers.Dense(5, activation ="relu",name="dense1")(hidden_layer)
#     output_layer = keras.layers.Dense(2, activation="sigmoid", name ="dense2")(hidden_layer)
    
    
#     ### higher level vinalla neural network
#     input2 = keras.layers.Input(shape = (3))
#     concat = keras.layers.Concatenate()([output_layer, input2])
#     h2 = keras.layers.Dense(10,activation = "relu")(concat)
#     out2 = keras.layers.Dense(11, activation = "linear")(h2)
     
#     model = keras.Model(inputs = [input_layer, input2], outputs = [out2])
#     return model



In [43]:
class trade_env():
    def __init__(self, data,init_inventory = 100, time_range = 10, look_back = 20):
        self.env_data = data
        self.inventory = init_inventory
        self.time_remain = time_range # todo: time unit
        self.look_back = look_back # linked to model
        self.current_time = look_back
        self.current_price = None
        
        self.action_dir = {
            0: 0,
            1: 10,
            2: 20,
            3: 30,
            4: 40,
            5: 50,
            6: 60,
            7: 70,
            8: 80,
            9: 90,
            10: 100,
               
            
        }
        
        
        
    def reset(self):
        
        self.inventory = 100
        self.time_remain = 10
        

    
    
    def start(self):
        """
        return the beginin of state: 1. time_series state; 2. current state
    
        """
        # the original shape is num_of_security(feature) * num_of_price_attr(e.g. close, volume,etc) *time_steps
        # reshape the original data into shape of  num_of_attr*time_steps* num_of_securities
        time_series_input = self.env_data[:,:,self.current_time-self.look_back: self.current_time]
        time_series_input = np.moveaxis(time_series_input, [1,0], [0,2])  
        # current input consisits of current_price, inventory, and time left
        self.current_price = time_series_input[:,-1,0].ravel()[0] # todo: the 1st is the security that we want to trade
        current_input = np.array([self.current_price, self.inventory,self.time_remain])[None,:]
        
        
        return [time_series_input,current_input]
    
    def step(self, action_choosen):
        # reward based on previous observed price and action
        
        action = self.action_dir[int(action_choosen)]
        reward = self.current_price*np.minimum(action, self.inventory)
        end_of_epsoide = False
        
       
        
        if self.time_remain > 0: # if the previous action is executed with remaining time larger than 0
            
             # progress time and other variables # updates
            self.current_time = self.current_time+1
            self.inventory = np.maximum(self.inventory - action, 0)
            self.time_remain = self.time_remain -1
            
            if self.inventory>0:# the trade hasn't finish and there are still time to do so
                time_series_input = self.env_data[:,:,self.current_time-self.look_back: self.current_time]
                time_series_input = np.moveaxis(time_series_input, [1,0], [0,2])  
                self.current_price = time_series_input[:,-1,0].ravel()[0] # todo: the 1st is the security that we want to trade
                current_input = np.array([self.current_price, self.inventory,self.time_remain])[None,:]
                return [time_series_input,current_input], reward, end_of_epsoide
           
            else: # trade succssfully executed and get the reward as previous step
                end_of_epsoide = True   
                return None, reward, end_of_epsoide
        
        else: # failed to liquid the security within the target time, get a large negative punishment
            
            reward = -1000
            end_of_epsoide = True
            
            return None, reward, end_of_epsoide
                
            

In [50]:
main_model = trade_model()
target_model = trade_model()
loss_function = keras.losses.Huber()
optimizer = keras.optimizers.Adam(learning_rate= 0.1)
num_episode = 300
gamma = 0.98
epi_sode_idx = 0
# main_model.summary()
# tf.keras.utils.plot_model(
#     main_model, to_file='model.png', show_shapes=True,
#     show_layer_names=True, rankdir='TB', expand_nested=True, dpi=60
# )


In [58]:
ep = 0
env = trade_env(data = train_data[:,:,ep+1:])
env.inventory = 100
env.time_remain = 10
state1 = env.start()
main_model.predict(state1)

array([[806.5511  , 549.78174 , 961.2976  , 256.65253 , 177.26773 ,
        191.80092 , 181.39362 ,  93.35716 ,  48.996853,  64.33158 ,
        111.90408 ]], dtype=float32)

In [52]:
np.random.seed(777)
for ep in range(num_episode):

    
    env = trade_env(data = train_data[:,:,ep+1:])
    current_state = env.start()
    while True:
    
        
        z = np.random.uniform()
        if z<=0.9:
            action = tf.argmax(main_model(current_state),axis=1).numpy()[0]
        else:
            action = np.random.choice(11)
            
        next_state, reward, end_episode = env.step(action)
        masks = tf.one_hot(action, 11)
        
#         print(current_state[1],next_state[1])
        future_reward = None
        
        if end_episode:
            G = reward
        else:
            future_reward = gamma*tf.reduce_max(main_model.predict(next_state),axis=1)
            G = reward + future_reward # todo: should use target
#         if future_reward is None:
        print("oops",action,  current_state[1], reward )
            
 
        
            
            
        with tf.GradientTape() as tape:
            
#             tape.watch(main_model.trainable_variables)
            current_state_action_val = main_model(current_state)
            current_predict = tf.reduce_sum(tf.multiply(current_state_action_val,masks),axis=1)
    
    
#             current_predict = tf.reduce_max(current_state_action_val,axis=1)

            loss = loss_function(G, current_predict)
            grad = tape.gradient(loss, main_model.trainable_variables)
            optimizer.apply_gradients(zip(grad, main_model.trainable_variables))
       
 
#         if main_model.predict(current_state)[0][action]==0:
#             print(grad)
        print("______________________________________")
            
        current_state = next_state
        
        if end_episode:
            break
    print("*********************************************")
        
        

oops 2 [[  1.19432762 100.          10.        ]] 23.886552462702827
______________________________________
oops 2 [[ 1.2013739 80.         9.       ]] 24.027478016353612
______________________________________
oops 2 [[ 1.20969459 60.          8.        ]] 24.193891855670525
______________________________________
oops 2 [[ 1.20720332 40.          7.        ]] 24.144066368207064
______________________________________
oops 2 [[ 1.21702453 20.          6.        ]] 24.340490608388933
______________________________________
*********************************************
oops 10 [[  1.2013739 100.         10.       ]] 120.13739008176807
______________________________________
*********************************************
oops 0 [[  1.20969459 100.          10.        ]] 0.0
______________________________________
oops 10 [[  1.20720332 100.           9.        ]] 120.72033184103532
______________________________________
*********************************************
oops 10 [[  1.20720332 100.  

______________________________________
oops 10 [[ 1.21866894 30.          9.        ]] 36.56006807838881
______________________________________
*********************************************
oops 4 [[  1.21866894 100.          10.        ]] 48.74675743785175
______________________________________
oops 4 [[ 1.22439969 60.          9.        ]] 48.975987568294215
______________________________________
oops 4 [[ 1.24667727 20.          8.        ]] 24.933545461646297
______________________________________
*********************************************
oops 4 [[  1.22439969 100.          10.        ]] 48.975987568294215
______________________________________
oops 4 [[ 1.24667727 60.          9.        ]] 49.867090923292594
______________________________________
oops 4 [[ 1.22703485 20.          8.        ]] 24.54069698128255
______________________________________
*********************************************
oops 4 [[  1.24667727 100.          10.        ]] 49.867090923292594
_______________

oops 3 [[ 1.24363923 10.          7.        ]] 12.43639233871465
______________________________________
*********************************************
oops 3 [[  1.22325683 100.          10.        ]] 36.697704820985905
______________________________________
oops 3 [[ 1.24614284 70.          9.        ]] 37.384285238582684
______________________________________
oops 3 [[ 1.24363923 40.          8.        ]] 37.30917701614395
______________________________________
oops 3 [[ 1.22948501 10.          7.        ]] 12.294850132991296
______________________________________
*********************************************
oops 3 [[  1.24614284 100.          10.        ]] 37.384285238582684
______________________________________
oops 3 [[ 1.24363923 70.          9.        ]] 37.30917701614395
______________________________________
oops 3 [[ 1.22948501 40.          8.        ]] 36.88455039897389
______________________________________
oops 3 [[ 1.23188996 10.          7.        ]] 12.318899563821432


oops 8 [[ 1.20900394 20.          9.        ]] 24.18007884924501
______________________________________
*********************************************
oops 8 [[  1.20900394 100.          10.        ]] 96.72031539698004
______________________________________
oops 8 [[ 1.20324852 20.          9.        ]] 24.06497046236572
______________________________________
*********************************************
oops 8 [[  1.20324852 100.          10.        ]] 96.25988184946289
______________________________________
oops 8 [[ 1.19616936 20.          9.        ]] 23.923387146504197
______________________________________
*********************************************
oops 8 [[  1.19616936 100.          10.        ]] 95.69354858601679
______________________________________
oops 8 [[ 1.17521963 20.          9.        ]] 23.50439261826359
______________________________________
*********************************************
oops 8 [[  1.17521963 100.          10.        ]] 94.01757047305436
__________

oops 0 [[ 1.09246081 90.          9.        ]] 0.0
______________________________________
oops 0 [[ 1.09160572 90.          8.        ]] 0.0
______________________________________
oops 5 [[ 1.09928509 90.          7.        ]] 54.96425473486016
______________________________________
oops 9 [[ 1.11066027 40.          6.        ]] 44.42641079716669
______________________________________
*********************************************
oops 0 [[  1.09246081 100.          10.        ]] 0.0
______________________________________
oops 0 [[  1.09160572 100.           9.        ]] 0.0
______________________________________
oops 0 [[  1.09928509 100.           8.        ]] 0.0
______________________________________
oops 0 [[  1.11066027 100.           7.        ]] 0.0
______________________________________
oops 8 [[  1.10440331 100.           6.        ]] 88.35226455196116
______________________________________
oops 3 [[ 1.09028197 20.          5.        ]] 21.805639488754323
_____________________

...

______________________________________
oops 2 [[ 0.9272145 40.         7.       ]] 18.54429000735872
______________________________________
oops 2 [[ 0.94749413 20.          6.        ]] 18.94988263055552
______________________________________
*********************************************
oops 2 [[  0.93806758 100.          10.        ]] 18.761351536902517
______________________________________
oops 2 [[ 0.95168325 80.          9.        ]] 19.033665092148375
______________________________________
oops 8 [[ 0.9272145 60.         8.       ]] 55.63287002207615
______________________________________
*********************************************
oops 2 [[  0.95168325 100.          10.        ]] 19.033665092148375
______________________________________
oops 5 [[ 0.9272145 80.         9.       ]] 46.36072501839679
______________________________________
oops 2 [[ 0.94749413 30.          8.        ]] 18.94988263055552
______________________________________
oops 2 [[ 0.92473967 10.          7. 

oops 2 [[ 0.89995026 60.          8.        ]] 17.999005134656258
______________________________________
oops 2 [[ 0.90942614 40.          7.        ]] 18.188522871625374
______________________________________
oops 2 [[ 0.90923293 20.          6.        ]] 18.18465851863728
______________________________________
*********************************************
oops 2 [[  0.92464512 100.          10.        ]] 18.492902334644747
______________________________________
oops 2 [[ 0.89995026 80.          9.        ]] 17.999005134656258
______________________________________
oops 2 [[ 0.90942614 60.          8.        ]] 18.188522871625374
______________________________________
oops 2 [[ 0.90923293 40.          7.        ]] 18.18465851863728
______________________________________
oops 2 [[ 0.90234287 20.          6.        ]] 18.046857335487505
______________________________________
*********************************************
oops 2 [[  0.89995026 100.          10.        ]] 17.99900513465625

______________________________________
oops 2 [[ 0.96436573 40.          7.        ]] 19.287314644664523
______________________________________
oops 2 [[ 0.97154767 20.          6.        ]] 19.430953467434605
______________________________________
*********************************************
oops 2 [[  0.92550432 100.          10.        ]] 18.510086372400295
______________________________________
oops 2 [[ 0.96226089 80.          9.        ]] 19.24521786317724
______________________________________
oops 2 [[ 0.96436573 60.          8.        ]] 19.287314644664523
______________________________________
oops 2 [[ 0.97154767 40.          7.        ]] 19.430953467434605
______________________________________
oops 2 [[ 0.986002 20.        6.      ]] 19.720039959054304
______________________________________
*********************************************
oops 2 [[  0.96226089 100.          10.        ]] 19.24521786317724
______________________________________
oops 2 [[ 0.96436573 80.       

______________________________________
oops 2 [[ 1.01375956 80.          9.        ]] 20.275191264917844
______________________________________
oops 2 [[ 1.02151294 60.          8.        ]] 20.430258706099508
______________________________________
oops 2 [[ 1.0158644 40.         7.       ]] 20.31728804640512
______________________________________
oops 2 [[ 1.01146151 20.          6.        ]] 20.22923013044247
______________________________________
*********************************************
oops 2 [[  1.01375956 100.          10.        ]] 20.275191264917844
______________________________________
oops 8 [[ 1.02151294 80.          9.        ]] 81.72103482439803
______________________________________
*********************************************
oops 2 [[  1.02151294 100.          10.        ]] 20.430258706099508
______________________________________
oops 2 [[ 1.0158644 80.         9.       ]] 20.31728804640512
______________________________________
oops 2 [[ 1.01146151 60.         

______________________________________
oops 2 [[ 1.07300749 60.          8.        ]] 21.460149887563773
______________________________________
oops 2 [[ 1.05146989 40.          7.        ]] 21.029397859806206
______________________________________
oops 2 [[ 1.02878532 20.          6.        ]] 20.575706374949124
______________________________________
*********************************************
oops 2 [[  1.07243606 100.          10.        ]] 21.448721269152188
______________________________________
oops 2 [[ 1.07300749 80.          9.        ]] 21.460149887563773
______________________________________
oops 2 [[ 1.05146989 60.          8.        ]] 21.029397859806206
______________________________________
oops 2 [[ 1.02878532 40.          7.        ]] 20.575706374949124
______________________________________
oops 2 [[ 1.02333 20.       6.     ]] 20.46660006824283
______________________________________
*********************************************
oops 2 [[  1.07300749 100.          

# test

In [ ]:
## genearte fake inputs 
from scipy.stats import norm
sample_size, time_lenght, ts_feature_num = 1000,60,5
current_feature_num = 3

time_series_data = norm.rvs( size = [sample_size, time_lenght, ts_feature_num])
current_data = norm.rvs(size=[sample_size, current_feature_num])
target_data = norm.rvs(size = [sample_size,10])


In [ ]:
main_model.predict([time_series_data[:1,:,:], current_data[:1,:]])

In [ ]:
actions_hist = []
state_hist = []
next_state_hist = []
reward_hist = []
gamma = 0.98
epsilon = 0.05
env = fake_environment()

    # replay buffer 
#     actions_hist.append(current_action)
#     state_hist.append(current_state)
#     reward_hist.append(reward)
#     next_state_hist.append(next_state)
    
# a single episode:

while 1:

    current_state = [time_series_data[:1,:,:], current_data[:1,:]]
    current_action = tf.reduce_max(main_model.predict(current_state),axis=1)
    next_state,reward,done = next(env) # next(env(current_action,current_action))
    if done:
        G = reward
    else:
        G = reward + tf.reduce_max(target_model.predict(next_state),axis=1)

    with tf.GradientTape() as tape:
        
        tape.watch(main_model.trainable_variables)
        current_predicted_act_values = main_model(current_state)
        current_state_value = tf.reduce_max(current_predicted_act_values,axis=1)
        loss = loss_function(G,current_state_value)
        grad = tape.gradient(loss, main_model.trainable_variables)
        optimizer.apply_gradients(zip(grad,main_model.trainable_variables))
    
    current_state = next_state
    if done:
        break
        

    



In [ ]:
## genearte fake inputs 
from scipy.stats import norm
sample_size, time_lenght, ts_feature_num = 1000,60,3
current_feature_num = 5

time_series_data = norm.rvs( size = [sample_size, time_lenght, ts_feature_num])
current_data = norm.rvs(size=[sample_size, current_feature_num])
target_data = norm.rvs(size = [sample_size,10])

def fake_environment():
    idx = 1
    while 1:
        if idx%5 ==0:
            yield None,0.01, True
        else:
            yield  [time_series_data[idx-1:idx,:,:], current_data[idx-1:idx,:]],0.01, False
        idx = idx+1
        


In [ ]:
# perform 1 step gradient update on the model
optimizer = keras.optimizers.RMSprop()

In [ ]:
env = fake_environment()

In [ ]:
current_state = [time_series_data[:0,:,:], current_data[:0,:]]
# current_action = tf.reduce_max(main_model.predict(current_state),axis=1)

In [ ]:
actions_hist = []
state_hist = []
next_state_hist = []
reward_hist = []
gamma = 0.98
epsilon = 0.05
env = fake_environment()

    # replay buffer 
#     actions_hist.append(current_action)
#     state_hist.append(current_state)
#     reward_hist.append(reward)
#     next_state_hist.append(next_state)
    
# a single episode:

while 1:

    current_state = [time_series_data[:1,:,:], current_data[:1,:]]
    current_action = tf.reduce_max(main_model.predict(current_state),axis=1)
    next_state,reward,done = next(env) # next(env(current_action,current_action))
    if done:
        G = reward
    else:
        G = reward + tf.reduce_max(target_model.predict(next_state),axis=1)

    with tf.GradientTape() as tape:
        
        tape.watch(main_model.trainable_variables)
        current_predicted_act_values = main_model(current_state)
        current_state_value = tf.reduce_max(current_predicted_act_values,axis=1)
        loss = loss_function(G,current_state_value)
        grad = tape.gradient(loss, main_model.trainable_variables)
        optimizer.apply_gradients(zip(grad,main_model.trainable_variables))
    
    current_state = next_state
    if done:
        break
        

    

